In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import style
import tables
import copy
import yaml

# the config file holds paths to the hdf5 files (in this example)
cfg_file = 'config.yaml'
cfg_file = yaml.safe_load(open(cfg_file))

In [ ]:
# we need to setup a GZK flux to weight to
# see docs for use: https://github.com/clark2668/ehefluxes
from functools import partial
from ehefluxes import fluxes
gzk_flux = fluxes.EHEFlux("cosmogenic_ahlers2010_1E18") # choose a model and instiate it
gzk_partial = partial(gzk_flux, which_species="nue_sum") # nue_sum = nue + nuebar

# make a livetime assumption
livetime = 365 * 24 * 60 * 60

### How to harvest weights from juliet files


In [ ]:
import weights_helper as weighting

# empty arrays to hold weights and charge
ehe_weights = np.asarray([])
ehe_charge = np.asarray([])
ehe_energy_weights = None # for plotting in true space

# these are for calculating effective areas; one per flavor
nu_areas_sum = {
    "nue": None,
    "numu": None,
    "nutau": None
}
energy_bins, energy_bin_centers = weighting.get_juliet_enu_binning()

In [ ]:

# juliet comes in five injected species, and two injected levels
# have to loop over all of them
juliet_species = ["nue", "numu", "nutau", "mu", "tau"]
juliet_energy_levels = ["high_energy", "very_high_energy"]
# juliet_species = ['nue']
# juliet_energy_levels = ["high_energy"]

# loop over all Juliet species
for s in juliet_species:
    for l in juliet_energy_levels:
            
        print(f"Working on juliet {s}, {l}")
        the_f = tables.open_file(cfg_file['juliet'][s][l]['file'])

        # get the variable of interest
        charge = the_f.get_node(f'/Homogenized_QTot').col(f'value')
        ehe_charge = np.concatenate((ehe_charge, charge))
        # and some truth variables
        energies = the_f.get_node('/I3JulietPrimaryParticle').col('energy')

        # get weighting information
        weight_dict, prop_matrix, evts_per_file = weighting.get_juliet_weightdict_and_propmatrix(the_f)
        evts_per_file = weighting.correct_events_per_file(s, l)
        n_gen = cfg_file['juliet'][s][l]['n_files'] * evts_per_file
    
        # calculate the weights per event for this flux model; this takes some time!
        # We need to convolve over the propagation matrices contining earth absorption
        # and primary flux -> observable particle conversion.
        weights = weighting.calc_juliet_weight_from_weight_dict_and_prop_matrix(
            weight_dict=weight_dict, prop_matrix=prop_matrix, 
            flux=gzk_partial, n_gen=n_gen, livetime=livetime
        )
        ehe_weights = np.concatenate((ehe_weights, weights))
        
        # We can also project this through into true neutrino energy space.
        # Remember, a juliet event is injected *at the detector*.
        # So a single juliet event contributes to many energy bins
        # in "true surface fluxes."
        # So we can't compute "which bin" an event lives in in "true" space.
        # It only works as an ensemble.
        # So the output, "the_enu_weights", has length "number of true neutrino energy bins".
        # This is unlike the method above, which can return a rate for that event in observable space.
        # Confused yet? :) 
        the_enu_weights = weighting.calc_juliet_flux_weight(
            weight_dict = weight_dict, prop_matrix = prop_matrix, 
            flux= gzk_partial, n_gen = n_gen, livetime=livetime,
        )
        if ehe_energy_weights is None:
            ehe_energy_weights = copy.deepcopy(the_enu_weights)
        else:
            ehe_energy_weights += copy.deepcopy(the_enu_weights)
        
        
        # finally we can calculate effective area
        # each injected type (nue, numu, nutau, mu, tau)
        # comes with a 3 dimensional matrix specifying it's contribution
        # to the *surface* nue, numu, and nutau flux
        # so we have to loop over that
        for iN, n in enumerate(nu_areas_sum.keys()):
            print("  Working on neutrino species {}".format(n))
            prop_i = prop_matrix[iN]
            area = weighting.calc_juliet_effective_area(
                energies = energies, weight_dict=weight_dict, n_gen = n_gen,
                energy_bins = energy_bins,  prop_matrix=prop_i,
            )
            if nu_areas_sum[n] is None:
                nu_areas_sum[n] = copy.deepcopy(area)
            else:
                nu_areas_sum[n] += copy.deepcopy(area)
            del area

        the_f.close()

### Plots in observable space

In [ ]:
# plot up our observable space
fig, ax = plt.subplots(1, 1, figsize=(7,5))
charge_bins = np.logspace(4, 8, 31)
ax.hist(ehe_charge, weights=ehe_weights, 
        bins=charge_bins, linewidth=3, histtype='step'
        )
ax.set_ylabel("Events / {:.2f} days".format(livetime/60/60/24))
ax.set_xlabel("HQtot [PE]")
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim([1E-6, 1E-1])


### Plots in True Neutrino Space

In [ ]:
fig2, ax2 = plt.subplots(1, 1 , figsize=(7,5))
ax2.hist(energy_bin_centers, bins=energy_bins,
        weights=ehe_energy_weights, histtype='step', linewidth=3
        )
ax2.set_ylabel("Events / {:.2f} days".format(livetime/60/60/24))
ax2.set_xlabel(r'Energy [GeV]')
ax2.set_xscale('log')
ax2.set_yscale('log')

### Plots of effective areas

In [ ]:
fig3, ax3 = plt.subplots(1, 1 , figsize=(7,5))
for iN, n in enumerate(nu_areas_sum.keys()):
        # sum along axis=0 to sum over all the zenith bins
        ax3.plot(energy_bin_centers, nu_areas_sum[n].sum(axis=0),
                label=f"{n}"
                )
ax3.set_ylabel(r"Effective Area [m$^2$]")
ax3.set_xlabel(r'Neutrino Energy [GeV]')
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlim([1E5, 1E11])
ax3.legend()